---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Have to specify entries are separated by new line. Setting header = None stops the first row entry from becoming the column name.
    uni_towns = pd.read_csv('university_towns.txt',sep='\n', header=None, names=['RegionName'])
    #Enter state name in column 'State' in the same row as 'RegionName'. Set everything else to NaN.
    uni_towns['State'] = np.where(uni_towns['RegionName'].str.contains('edit'),uni_towns['RegionName'],np.NaN)
    # Replace NaN values in State using Forward fill
    uni_towns['State'].fillna(method='ffill',inplace=True)
    # Rearrange columns to required order: first State and then RegionName
    uni_towns = uni_towns[['State','RegionName']]
    # Use '(' and '[' split for both columns. expand = True splits the string into different columns.
    for col in uni_towns:
        uni_towns[col] = uni_towns[col].str.split('[', expand=True)[0].str.split('(',expand=True)[0].str.rstrip()
    # Remove rows where State=RegionName
    uni_towns = uni_towns[uni_towns['State'] != uni_towns['RegionName']]  
    
    return uni_towns

In [6]:
def read_gdp():
    
    # Read and skip irrelevant rows    
    GDP = pd.read_excel('gdplev.xls', skiprows=5)
    # Drop irrelevant columns
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]],axis=1)
    # Rename columns
    GDP.columns = ['Quarter','GDP']
    # Find index of first relevant Q (2020q1) and drop every row before it
    GDP.index[GDP['Quarter'] == '2000q1']
    GDP = GDP.drop(GDP.index[0:214])
    # Reset index so it starts at 0 and drop original index
    GDP.reset_index(drop=True, inplace=True)
    # New column with difference between successive row values of 'GDP'
    GDP['GDP Diff'] = GDP['GDP'].diff(periods=1)

    return GDP

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = read_gdp()
    # New GDP_decrease dataframe only with quarters with GDP decrease
    GDP_decrease = GDP[GDP['GDP Diff']<0]
    # Get index values into a column to use diff()
    GDP_decrease['Index'] = GDP_decrease.index
    GDP_decrease['Index Diff'] = GDP_decrease['Index'].diff()
    # Find the FIRST quarter where index difference is 1. idxmin() gives the first occurence of the minimum value.
    min_index = GDP_decrease['Index Diff'].idxmin()
    #The recesssion start quarters are those that come immediately before the quarters with 'Index Diff' = 1
    
    return GDP['Quarter'].iloc[min_index-1]

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = read_gdp()
    # Finds all the quarters where there is a rise:
    GDP_increase = GDP[GDP['GDP Diff']>0]
    # Find the first quarter after the recession starts where there is a successive rise:
    GDP_increase['Index'] = GDP_increase.index
    GDP_increase['Index Diff'] = GDP_increase['Index'].diff()
    # Get index of the "recovery quarter" (first quarter positive after at least 2 quarters of negative GPD)
    index_of_recovery = GDP_increase[GDP_increase['Index Diff']>=3].index.item()
    # The end of the recession is the next one to the "recovery quarter"
    return GDP['Quarter'].iloc[index_of_recovery+1]

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = read_gdp()
    # Get indices of recession start and end quarters:
    start_recession_index = GDP.loc[GDP['Quarter'] == get_recession_start()].index.item()
    end_recession_index = GDP.loc[GDP['Quarter'] == get_recession_end()].index.item()
    # Reduce GDP to the recession range
    Recession = GDP.iloc[start_recession_index:end_recession_index+1]
    # Get index of bottom quarter
    bottom_recession_index = Recession['GDP'].idxmin()
    
    return Recession['Quarter'][bottom_recession_index]

In [13]:
def months_to_Q(date: str):
    date = date.split('-')
    month = int(date[1])
    quarter = int((month - 1) / 3) + 1
    return date[0] + 'q' + str(quarter)

In [14]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')

    # Drop columns with unnecessary data
    columns_to_delete = all_homes.ix[:, '1996-04':'1999-12'].columns
    all_homes = all_homes.drop(columns_to_delete, axis = 1)
    columns_to_delete = all_homes.columns[[0,3,4,5]]
    all_homes = all_homes.drop(columns_to_delete,axis=1)
    
    # Use given dictionary to change States acronyms to their names
    all_homes['State'] = all_homes['State'].map(states)
    
    # Change order of columns State and RegionName for multiindex
    columns = all_homes.columns.tolist()
    columns_unchanged = columns[2:]
    columns_ordered = ['State', 'RegionName']+columns_unchanged
    all_homes = all_homes[columns_ordered]
    
    ## Set index and sort it alphabetically
    all_homes = all_homes.set_index(['State','RegionName']).sort_index()
        
    # Use function to change columns (yyyy-mm) to quarters and then group by them to calculate mean
    all_homes = all_homes.groupby(months_to_Q, axis=1).mean()
    
    return all_homes

In [15]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Get housing data
    housing_data = convert_housing_data_to_quarters()
    
    # Get indexes of recession start Q and recession bottom Q
    before_recession_start_idx = housing_data.columns.get_loc(get_recession_start())-1
    recession_bottom_idx = housing_data.columns.get_loc(get_recession_bottom())
    
    # Add ratio column between the Q before the recession start and the recession bottom Q
    housing_data['price_ratio'] = housing_data[housing_data.columns[before_recession_start_idx]]/housing_data[housing_data.columns[recession_bottom_idx]]
    # Keep only quarter before recession, bottom of recession and price_ratio column
    housing_data = housing_data[[before_recession_start_idx,recession_bottom_idx,-1]]
    housing_data =  housing_data.dropna()
    
    # Get university towns dataframe and set 'State' and 'RegionName' as index
    uni_towns = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # Add column indicating if it is a university town or not
    housing_data['university_town'] = housing_data.index.isin(uni_towns.index)
    UT = housing_data[housing_data['university_town']==True]
    Non_UT = housing_data[housing_data['university_town']==False]
    
    # Perform Ttest and get its p-value
    p_value = ttest_ind(UT['price_ratio'], Non_UT['price_ratio']).pvalue
    p_value
    
    if p_value < 0.01:
        different=True
    else:
        different=False
        
    # Better depending on which one is LOWER! Remember prices go up during a recession so lower is better.
    if UT['price_ratio'].mean() < Non_UT['price_ratio'].mean():
        better='university town'
    else:
        better='non-university town'
    
    return (different, p_value, better)